In [1]:
from pathlib import Path
import pandas as pd
import time
import os
import tqdm

In [2]:
# Replace 'your_file.csv' with the actual path to your CSV file
input_fp = Path.cwd() / 'tidy_data' # path of files to be found

output_fp = Path.cwd() / 'unique_data' # output path of files generated
# Use the Path object to actually create the subfolder
Path.mkdir(output_fp, exist_ok=True)

In [3]:
def find_matching_rows(df):
    grouped = df.groupby(['date_of_sale', 'purchase_amount', 'zip_code', 'housing_type'])
    matching_indices = []

    for _, group in grouped:
        if len(group) > 1:
            if group['housing_type'].iloc[0] == 'Ejerlejlighed':
                address_prefix = group['address'].str[:3]
                if address_prefix.nunique() == 1:
                    matching_indices.extend(group.index.tolist())
            else:
                matching_indices.extend(group.index.tolist())

    return matching_indices

In [4]:
start_time = time.time()

# Iterate through log files in the folder
for filename in sorted(os.listdir(input_fp)): #Alterntive: for filename in tqdm.tqdm(sorted(os.listdir(input_fp))):
    if filename.startswith('tidy_sales_1992_2022_') and filename.endswith('.csv'):
        df = pd.read_csv(input_fp/filename, low_memory=False) # read the CSV file into a DataFrame
        #print(df.shape)
        matching_indices = find_matching_rows(df)
        matching_df = df[df.index.isin(matching_indices)]
        unique_matching_indices = list(set(matching_indices))

        # List of index numbers to exclude
        index_numbers_to_exclude = unique_matching_indices
        
        # Create a new DataFrame excluding rows with specified index numbers
        new_df = df[~df.index.isin(index_numbers_to_exclude)]
        
        # Rename and save the new DataFrame to a CSV file without index numbers
        output_filename = filename.replace('tidy_', 'unique_')  # Replace 'tidy_' with 'unique_' in filename
        new_df.to_csv(f'{output_fp}/{output_filename}', index=False)

        print(f'Number of rows in {filename}: {len(df)}')
        print(f'Number of matching rows: {len(matching_df)}')
        print(f'Number of unique rows: {len(new_df)}')
        print()

elapsed_time = time.time() - start_time
print(f'Runtime: {elapsed_time}')

Number of rows in tidy_sales_1992_2022_101.csv: 297624
Number of matching rows: 57596
Number of unique rows: 240028

Number of rows in tidy_sales_1992_2022_147.csv: 56327
Number of matching rows: 10666
Number of unique rows: 45661

Number of rows in tidy_sales_1992_2022_151.csv: 18191
Number of matching rows: 5011
Number of unique rows: 13180

Number of rows in tidy_sales_1992_2022_153.csv: 9457
Number of matching rows: 1751
Number of unique rows: 7706

Number of rows in tidy_sales_1992_2022_155.csv: 8115
Number of matching rows: 536
Number of unique rows: 7579

Number of rows in tidy_sales_1992_2022_157.csv: 56459
Number of matching rows: 10803
Number of unique rows: 45656

Number of rows in tidy_sales_1992_2022_159.csv: 30578
Number of matching rows: 3960
Number of unique rows: 26618

Number of rows in tidy_sales_1992_2022_161.csv: 9207
Number of matching rows: 1129
Number of unique rows: 8078

Number of rows in tidy_sales_1992_2022_163.csv: 7311
Number of matching rows: 439
Number o